# Surebets

Situation where you can profit regardless of a match outcome based on different bookmakers providing different odds

If (1/odds1) + (1/odds2) <1.0, you've got a sure bet


| Bookie | Over     | Under     |
|--------|----------|-----------|
|   1    |   **2**  |   1.9     |
|   2    |  1.8     |  **2.1**  |


With $100 to bet. $51.21 on the over with bookie 1, $48.78 on the under with bookie 2

**Scenario 1 - Over**
You win on Bookmaker 1 but lose on Bookmaker 2
`Profit 1= 51.21*2 — 51.21 — 48.78 = 2.43`

**Scenario 2 - Under**
You win on Bookmaker 2 but lose on Bookmaker 1
`Profit 2 = 48.78*2.1 — 51.21 — 48.78= 2.44`

In [3]:
from selenium import webdriver
import pandas as pd
import time

In [1]:
def find_surebet(odds1, odds2):
    sum_odds = (1/odds1) + (1/odds2)
    if sum_odds<1:
        message = 'Surebet Found!'
    else:
        message = 'Keep scraping'
    return message

In [12]:
!xattr -d com.apple.quarantine /Users/jaredgodar/codeup-data-science/surebet/chromedriver


In [13]:
#Writing our First Selenium Python Test
web = 'https://sports.tipico.de/en/all/football/spain/la-liga' #you can choose any other league (update 1)
path =  '/Users/jaredgodar/codeup-data-science/surebet/chromedriver'
driver = webdriver.Chrome(path)
driver.get(web)


/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/1413878318.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [19]:
#Make ChromeDriver click a button
#time.sleep(5) #add implicit wait, if necessary
#accept = driver.find_element_by_xpath('//*[@id="_evidon-accept-button"]')
#accept.click()

In [15]:
#Initialize  storage
teams = []
x12 = [] #3-way
odds_events = []

In [16]:
#select only upcoming matches box
box = driver.find_element_by_xpath('//div[contains(@testid, "Program_SELECTION")]') #update 3
#Looking for 'sports titles'
sport_title = box.find_elements_by_class_name('SportTitle-styles-sport')

/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/2537215173.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  box = driver.find_element_by_xpath('//div[contains(@testid, "Program_SELECTION")]') #update 3
/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:464: UserWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead")


In [17]:
for sport in sport_title:
    # selecting only football
    if sport.text == 'Football':
        parent = sport.find_element_by_xpath('./..') #immediate parent node
        grandparent = parent.find_element_by_xpath('./..') #grandparent node = the whole 'football' section
        #Looking for single row events
        single_row_events = grandparent.find_elements_by_class_name('EventRow-styles-event-row')
        #Getting data
        for match in single_row_events:
            #'odd_events'
            odds_event = match.find_elements_by_class_name('EventOddGroup-styles-odd-groups')
            odds_events.append(odds_event)
            # Team names
            for team in match.find_elements_by_class_name('EventTeams-styles-titles'):
                teams.append(team.text)
        #Getting data: the odds        
        for odds_event in odds_events:
            for n, box in enumerate(odds_event):
                rows = box.find_elements_by_xpath('.//*')
                if n == 0:
                    x12.append(rows[0].text)

driver.quit()
#Storing lists within dictionary
dict_gambling = {'Teams': teams, '1x2': x12}
#Presenting data in dataframe
df_gambling = pd.DataFrame.from_dict(dict_gambling)
print(df_gambling)

/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  warnings.warn("find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead")
/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:426: UserWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  warnings.warn("find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead")


                               Teams               1x2
0        Real Betis\nAthletic Bilbao  2.40\n3.00\n3.00
1           Real Sociedad\nCD Alaves  1.53\n3.80\n7.00
2           FC Barcelona\nCA Osasuna  1.32\n5.30\n8.50
3          RCD Mallorca\nReal Madrid  6.00\n4.30\n1.52
4         Athletic Bilbao\nCF Getafe  1.70\n3.50\n5.50
5              CD Alaves\nGranada CF  2.10\n3.30\n3.60
6              Elche CF\nCF Valencia  3.00\n3.20\n2.40
7             CA Osasuna\nUD Levante  2.00\n3.40\n3.70
8    Rayo Vallecano\nAtletico Madrid  3.70\n3.40\n2.00
9   Espanyol Barcelona\nRCD Mallorca  2.20\n3.30\n3.30
10           Cadiz CF\nVillarreal CF  5.00\n3.70\n1.70
11      RC Celta de Vigo\nReal Betis  2.40\n3.30\n2.90
12         Sevilla FC\nReal Sociedad  1.85\n3.50\n4.30
13         Real Madrid\nFC Barcelona  1.95\n3.70\n3.60


In [18]:
df_gambling

,Teams,1x2
0,Real Betis\nAthletic Bilbao,2.40\n3.00\n3.00
1,Real Sociedad\nCD Alaves,1.53\n3.80\n7.00
2,FC Barcelona\nCA Osasuna,1.32\n5.30\n8.50
3,RCD Mallorca\nReal Madrid,6.00\n4.30\n1.52
4,Athletic Bilbao\nCF Getafe,1.70\n3.50\n5.50
5,CD Alaves\nGranada CF,2.10\n3.30\n3.60
6,Elche CF\nCF Valencia,3.00\n3.20\n2.40
7,CA Osasuna\nUD Levante,2.00\n3.40\n3.70
8,Rayo Vallecano\nAtletico Madrid,3.70\n3.40\n2.00
9,Espanyol Barcelona\nRCD Mallorca,2.20\n3.30\n3.30
